In [1]:
# saves you having to use print as all exposed variables are printed in the cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# core libraries
import pandas as pd
import os
from pathlib import Path

%reload_ext autoreload
%autoreload 2
# for cleaning and discovery
from ds_discovery import TransitionAgent as Transition

# Set the environment Contract Path to use with the from_env() init factory method
os.environ['TR_CONTRACT_PATH'] = Path(os.environ['PWD'], 'contracts').as_posix()

import ds_discovery
print('DTU: {}'.format(ds_discovery.__version__))

DTU: 2.03.078


# Accelerated Machine learning
## Transitioning: Discovery and Observations
As part of the Accelerated ML discovery Vertical, Transitioning is a foundation base truth facilitating a **transparent** transition<br>
of the raw canonical dataset to a **fit-for-purpose** canonical dataset to enable the optimisation of discovery analysis and the identification of **features-of-interest**

With reference to the diagram and as a continuation of the Source Contract, this covers references: 
4. Transitioning Notebooks
5. Discovery Analysis and Information capture
6. Transitioning Contract Pipeline
7. Running Contract Pipeline and the Transitioned Canonical
![transition](../98_images/AccML-Transition.png)


### The Transitioning Notebook
One of the value accelerators of the Transitioning Contract it its ability to run across multiple Notebooks and multiple contributors as a single reference instance. This allows the contributor to play to Jupyters experimental strenghts while facilitating its weaknesses through controlled governace of activities and reference. 
* To retrieve the transitioning contract, we create a Transitioning instance using the unique contract name
* The transitioning object is a singletone instance and will load the current contract or create a new one if it doesn't exist


In [2]:
tr = Transition.from_env('synthetic_customer')

### Loading the source dataset
All data coming through the Accelerated ML vertical is now in __canonical__ form, be it source data, reference data, data dictionaries or value add information, <br>
In this case our __canonical__ is Pandas Dataframe, as the most familar canonical with Python and Data Scientists. We will see more of this later.


In [3]:
df = tr.load_source_canonical()

### Transitioning Discovery
Within the Transitioning instance is a number of discovery tools to help with visualising the Transitioning Contract. <br>
Within these are a set of reports
* Canonical Report
* Source Contract Report
* Cleaners Contract Report
* Notes Report

At this stage, of these, we are going to use the ``tr.canonical_report(df)`` to help us exainme the raw canonical<br>
Note: as all reports are also canonical in form, though have the option to be stylised, set as ``True by default``. 

We will see more of this later


#### Canonical Report 
The Canonical Report is a data dictionary of the canonical providing a reference view of the dataset's attribute properties, to allow for attribute typing and selection.
* Attribute: the current header name
* dType: the current data type of that attribute
* %_Null: the percentage of nulls data points observed
* %_Dom: the percentage perdominace of the primary occuring observation
* Count: the total count of data points not null
* Unique: the number of unique data points
* Observations: observational information about the datapoints dependant on their type

Here we have used the `stylised` option to help visualise the data with the red indicating depth of nulls or perdominant elements and blue potential canidates for catagoricals.

In [4]:
tr.canonical_report(df, stylise=True)

,Attribute,dType,%_Null,%_Dom,Count,Unique,Observations
0,Unnamed: 0,int64,0.0%,0.1%,1000,1000,max=999 | min=0 | mean=499.5
1,age,float64,15.0%,0.2%,850,837,max=89.46799999999999 | min=20.053 | mean=47.4
2,balance,float64,0.0%,0.2%,1000,976,max=873.97 | min=29.54 | mean=182.72
3,forename,object,0.0%,0.2%,1000,997,Sample: Salman | Edward | Edmund
4,gender,object,0.0%,62.6%,1000,2,Sample: M | F
5,id,object,0.0%,0.1%,1000,1000,Sample: CU_7930035 | CU_9086471 | CU_5814649
6,null,float64,100.0%,0.0%,0,0,max=nan | min=nan | mean=nan
7,online,int64,0.0%,80.7%,1000,2,max=1 | min=0 | mean=0.19
8,profession,object,10.0%,23.7%,900,15,Sample: Senior Sales Associate | Data Coordiator | Senior Editor
9,single cat,object,40.0%,100.0%,600,1,Sample: A


-------------
### Discovery Observations
#### Add any observations of the dataset
* Add an overview description of the dataset
* Include relevant information, such as the source system and any issues or problems
* Then add any observations for specific attributes that are noteworthy
* Notes on attributes should only relate to transitioning, that are not relevant to enrichment knowledge later in the processes

First we add any general notes about this contract

In [5]:
tr.add_notes(text='The file is a synthetic customer data file created for this demonstration')

#### Add the data source
It is good practice to also include the source of the data

In [6]:
tr.add_notes(label='source', text='This was generated using the Discovery Behavioral Synthetic Data Generator')
tr.add_notes(label='source', text='The script to rerun the data generation can be found in the synthetic scripts folder')

#### Attribution Observations
It is worth capturing observation where attributes might be removed or changed that are hidden from the transitioned view of data<br>
for example the `weight_cat` attribute has values, but it has a predomionant value that makes this column a likely candidtate for removal

In [7]:
tr.add_attribute_notes(attribute='null', text="Here for demo of removal of nulls")
tr.add_attribute_notes(attribute='weight_cat', text="Demonstration of removal of columns with predominant values")
tr.add_attribute_notes(attribute='weight_cat', text="the value 'A' is over 95% predominant")
tr.add_attribute_notes(attribute='start', text="changing this to start_date so it being a date is obvious")

### Create a report on the notes
We have asked for the notes to be stylised, this returns a style dataframe with elements blanked and formatted dates for presentation.<br>
removing this parameter returns a canonical dataframe.

Other note-worthy options are:
* note_type: (optional) the type of note to filter on, options are 'attribute', 'overview', 'dictionary'
* labels: (optional) s label or list of labels to filter on
*  date_format: (optional) the display date format
*  regex: a regular expression on the notes
*  re_ignore_case: if the regular expression should be case sensitive
*  drop_dates: (optional) excludes the 'date' column from the report

In [8]:
tr.report_notes(stylise=True)

,section,label,date,text
0,notes,comment,2019-12-23 11:24,The file is a synthetic customer data file created for this demonstration
1,,source,2019-12-23 11:24,This was generated using the Discovery Behavioral Synthetic Data Generator
2,,,2019-12-23 11:24,The script to rerun the data generation can be found in the synthetic scripts folder
3,attribute,null,2019-12-23 11:24,Here for demo of removal of nulls
4,,start,2019-12-23 11:24,changing this to start_date so it being a date is obvious
5,,weight_cat,2019-12-23 11:24,Demonstration of removal of columns with predominant values
6,,,2019-12-23 11:24,the value 'A' is over 95% predominant


-----------------
## Transitioning: Contract Pipeline
The clean methods are separated into two main types:
* Auto: Allows the auto selection and filtering of a complete dataset
* To: For data typing as a transitioning process into useable datatypes for feature discovery

They are static methods that can be used as tools, but here we use them to build our filter and typing intent as a contract pipeline.

In [9]:
tr.clean.__dir__()

['auto_clean_header',
 'auto_drop_duplicates',
 'auto_remove_columns',
 'auto_to_category',
 'filter_columns',
 'filter_headers',
 'list_formatter',
 'run_contract_pipeline',
 'to_bool_type',
 'to_category_type',
 'to_date_from_excel_type',
 'to_date_type',
 'to_float_type',
 'to_int_type',
 'to_numeric_type',
 'to_remove',
 'to_select',
 'to_str_type']

### Using the Cleaner Class methods
The class methods are static and by default return the typed or filtered DataFrame.<br>
All the typing method calls have common header selection parameters:
* **`headers`**: A string or list of column names. By default these will be selected unless you set the `drop` boolean to True
* **`dtypes`**: A string or list of data types (int, float, category, number, str, datetime), be=y default, these are selected unless `exclude` boolean is set to True
* **`regex`**: Use a regular expression to select columns.

In this example we are converting `start` to a datetime type.<br>
Note: Being a typed attribute the observations now change, giving max, min and mean rather than a sample.

In [10]:
df_typed = tr.clean.to_date_type(df, headers=['start'])
tr.canonical_report(df_typed, stylise=False).iloc[9:12]

,Attribute,dType,%_Null,%_Dom,Count,Unique,Observations
9,single cat,object,0.4,1.000,600,1,Sample: A
10,single num,float64,0.2,1.000,800,1,max=1.0 | min=1.0 | mean=1.0
11,start,datetime64[ns],0.0,0.007,1000,350,max=2018-12-30 00:00:00 | min=2018-01-01 00:00:00 | yr mean= 2018


### Extracting the Parameterised Intent
in order to create the **_contract pipeline_**, we are looking to extract the **_parameterised intent_** from the method.<br>
**_Parameterised intent_** is the recording of the intent and the specific parameters associated with that intent. In this instance:
* **Intent**: to convert a column to a date type
* **Parameterised**: to select the header `start`

Unlike the previous call, this time we set the parameter `inplace` to True. This then
* Changes the DataFrame passed with the _parameterised intent_
* Returns a recordeable _pipeline element_ representing the intent and its parameters

In [11]:
intent = tr.clean.to_date_type(df, headers=['start'], inplace=True)
intent

{'to_date_type': {'headers': ['start'],
  'drop': False,
  'as_num': False,
  'day_first': False,
  'year_first': False}}

### Adding the Intent to the Transitioning Pipeline
We can now take this **_intent_** and record it in our transitioning instance as a **_contract pipeline element_** using the `set_cleaner()` method

In [12]:
_ = tr.remove_cleaner()
tr.set_cleaner(intent)

... and that's it now recorded as part of our runnable **_contract pipeline_**.

In [13]:
tr.report_cleaners()

,level,intent,parameters
0,0,to_date_type,"headers=['start'], drop=False, as_num=False, day_first=False, year_first=False"


We can of course run it as a single command
> `tr.set_cleaner(tr.clean.to_date_type(df, headers=['start'], inplace=True))`

### Auto filters special case
The methods `auto_remove_columns` and `auto_to_category` have two different methods of recording **_intent_**
* An auto generated outcome that represents the actual parameterised intent
* The intent to auto filter. 

Here is an example:

* In the first instance we will generate the intent to auto remove
* We then reload the file and run it again with the parameter: `auto_contract = False`

In [14]:
tr.clean.auto_remove_columns(df, inplace=True)

{'auto_remove_columns': {'null_min': 0.998, 'predominant_max': 0.998}}

In [15]:
# reload the data source to return the missing columns
df = tr.load_source_canonical()

tr.clean.auto_remove_columns(df, auto_contract=False, inplace=True)

{'to_remove': {'headers': ['single num', 'null', 'single cat'], 'drop': False}}

As you see with the second output, the actual headers to be removed have been recorded as the intent. This allows flexibility in how we choose to control the auto filtering of incoming files though our contract pipeline.

The logic works like this:
* With the first example, each time we run our contract pipeline, the auto remove will remove **ANY** column that matches the auto remove criteria
* With the second example **ONLY** the columns identified in this discovery analysis will be removed. Therefore in a subsequent file, should another column appear that has null, constant or Quasi-constant values it will be passed through.
* The second example also allows us to discover what the Auto Remove has removed, allowing us to optimise the threshold values passed.

------------
## Transitioning: Selection, Filter and Typing
We can reset the cleaners, reload our data source, re-examine the Data Dictionary and start the process of transitioning the Dataset.


In [16]:
_ = tr.remove_cleaner()
df = tr.load_source_canonical()
tr.canonical_report(df)

,Attribute,dType,%_Null,%_Dom,Count,Unique,Observations
0,Unnamed: 0,int64,0.0%,0.1%,1000,1000,max=999 | min=0 | mean=499.5
1,age,float64,15.0%,0.2%,850,837,max=89.46799999999999 | min=20.053 | mean=47.4
2,balance,float64,0.0%,0.2%,1000,976,max=873.97 | min=29.54 | mean=182.72
3,forename,object,0.0%,0.2%,1000,997,Sample: Mya | Georgette | Louis
4,gender,object,0.0%,62.6%,1000,2,Sample: M | F
5,id,object,0.0%,0.1%,1000,1000,Sample: CU_9972454 | CU_7966084 | CU_9854891
6,null,float64,100.0%,0.0%,0,0,max=nan | min=nan | mean=nan
7,online,int64,0.0%,80.7%,1000,2,max=1 | min=0 | mean=0.19
8,profession,object,10.0%,23.7%,900,15,Sample: Analyst Programmer | Senior Sales Associate | Office Assistant II
9,single cat,object,40.0%,100.0%,600,1,Sample: A


### Tidy the headers
As good practice we clean the headers.
* This removes any hidden characters that sometimes lurk in the header name
* Replaces spaces in `single num` and `single cat` with underscore (use `replace_spaces` parameter to specify a different character) 
* Optionally set a case type for consitency across the headers (options are `lower`, `upper`, `title`)
* Optionally in this case we are also going to rename `start` to be `start_date` to identify it as a date

In [17]:
tr.set_cleaner(tr.clean.auto_clean_header(df, rename_map={'start': 'start_date'}, inplace=True))

### Auto remove selection
The `auto_remove_columns` and method quickly allows us to remove columns that contain poor quality data. We will remove columns with:
* More than 99% nulls
* Has a predominant value of more than 90% 

In addition we are going to pass in an extra list of considered null values, in this case **empty string**, to be considered as a null.

In [18]:
tr.set_cleaner(tr.clean.auto_remove_columns(df, null_min=0.99, predominant_max=0.90,inplace=True, nulls_list=['']))

### Auto categorise filter
The `auto_to_category` method allows us to quickly convert large columns of data into the useful Categorical data type.<br>
as setable parameters it considers
* The number of unique items in the columns
* A null value threshold so the unique numbers are not a result of poor data quantity

In [19]:
tr.set_cleaner(tr.clean.auto_to_category(df, unique_max=20, null_max=0.7, inplace=True))

### Category and Date typing
With most data transposition to a useful and usable dataset, the conversion of Dates and Categories is probably the most common.<br>
The methods `to_category_type` and `to_date_type` fullfil this.

In both methods we are using the `headers` parameter though you can also filter by `dtype` or a regex.

In [20]:
# Typing Catagories
tr.set_cleaner(tr.clean.to_category_type(df, headers=['gender', 'profession'], inplace=True))
# Typing Dates 
tr.set_cleaner(tr.clean.to_date_type(df, headers='start_date', day_first=True, inplace=True))


### Boolean typing
The `to_bool_type` allows us to specify a map of values to convert to true, with everything else considered false.

In [21]:
tr.set_cleaner(tr.clean.to_bool_type(df, bool_map={"?": 'Unknown'}, headers='online', inplace=True))

### Float, String Typing
Finally to tidy up our final types we run them through our typing methods to ensure they are fit for purpose.<br>
Here we have used the `dtype` parameter to capture all remaing columns of those types, and also set the precision of the `floats` to be 3

In [22]:
tr.set_cleaner(tr.clean.to_float_type(df, dtype=['float'], precision=3, inplace=True))

tr.set_cleaner(tr.clean.to_str_type(df, dtype=['object'], nulls_list=['', 'nan'], inplace=True))

### Integer Typing
It should be noted that we didn't convert `age` to an `int` as we are going to defer that up to the feature catalogue to make the decision on how to convert out the `nulls`. But we could have used the built in functions here to convert `age` and replace the `nulls` with an alternative value.

> `tr.clean.to_int_type(df, headers='age', fillna='mean' inplace=True)`

In the above code snippet we converted `age` from a `float`, replacing the `nulls`, that stopped it being an `int`, to the `mean` of the values.

This conversation should be made with care as you are hiding data changes from the feature cataloging. 


### Printing a Cleaners Report

We have asked for the notes to be stylised, this returns a style dataframe with elements blanked and formatted dates for presentation.<br>
removing this parameter returns a canonical dataframe.

Other note-worthy options are:
* `to_excel`: allowing a hard copy to be printed as an excel worksheet


In [23]:
tr.report_cleaners()

,level,intent,parameters
0,0,auto_clean_header,"rename_map={'start': 'start_date'}, replace_spaces=_"
1,,auto_remove_columns,"null_min=0.99, predominant_max=0.9, nulls_list=['']"
2,,auto_to_category,"null_max=0.7, unique_max=20"
3,,to_bool_type,"headers=online, drop=False, bool_map={'?': 'Unknown'}"
4,,to_category_type,"headers=['gender', 'profession'], drop=False"
5,,to_date_type,"headers=start_date, drop=False, as_num=False, day_first=True, year_first=False"
6,,to_float_type,"dtype=['float'], exclude=False, fillna=nan, errors=coerce, precision=3"
7,,to_str_type,"dtype=['object'], exclude=False, nulls_list=['', 'nan']"


### Changing the Cleaning Order
There will be circumstances where you wish to run certain cleaners in an order. This is easily achived by setting the cleaner level.

Lets imagine we want to ensure the `to_bool_type`, to fun last. To achieve this we move it up a level. Each level is run in order so level 1 will run after level 0 has completed.

In [24]:
tr.set_cleaner(tr.clean.to_bool_type(df, bool_map={1: True}, headers='online', inplace=True), level=1)
tr.report_cleaners()

,level,intent,parameters
0,0,auto_clean_header,"rename_map={'start': 'start_date'}, replace_spaces=_"
1,,auto_remove_columns,"null_min=0.99, predominant_max=0.9, nulls_list=['']"
2,,auto_to_category,"null_max=0.7, unique_max=20"
3,,to_bool_type,"headers=online, drop=False, bool_map={'?': 'Unknown'}"
4,,to_category_type,"headers=['gender', 'profession'], drop=False"
5,,to_date_type,"headers=start_date, drop=False, as_num=False, day_first=True, year_first=False"
6,,to_float_type,"dtype=['float'], exclude=False, fillna=nan, errors=coerce, precision=3"
7,,to_str_type,"dtype=['object'], exclude=False, nulls_list=['', 'nan']"
8,1,to_bool_type,"headers=online, drop=False, bool_map={1: True}"


As we see the `to_bool_type` has now been moved to level 1. and will be run after level 0. <br>
Note that by default any duplicate **_parameterised intent_** will be removed from other level but not **_intent_** with different **_patameters_** 

----------
## Transitioning: Finalise and Validate
### Persist the Canonical
We now have out typed, selected and filtered canonical ready for Feature Cataloging. From this we need to 
* Persist the canonical so it can be used for Feature Cataloging

Because the transitioning instance is managing governance and naming convention, we only have to pass the DataFrame and it does all the rest.

In [25]:
# save the clean file
tr.save_clean_canonical(df)

### Validation
We can validate our file has been saved by reloading it and checking all the correct typing and filtering has happened.

Again because we are using the transitioning instance we only need to call `load_canonical`, without parameters, to retrieve our canonical dataset

In [26]:
# check the results worked
df = tr.load_clean_canonical()
tr.canonical_report(df, stylise=False)

,Attribute,dType,%_Null,%_Dom,Count,Unique,Observations
0,Unnamed:_0,int64,0.00,0.001,1000,1000,max=999 | min=0 | mean=499.5
1,age,float64,0.15,0.002,850,837,max=89.46799999999998 | min=20.053 | mean=47.4
2,balance,float64,0.00,0.002,1000,976,max=873.97 | min=29.54 | mean=182.72
3,forename,object,0.00,0.002,1000,997,Sample: Macy | Holly | Isa
4,gender,category,0.00,0.626,1000,2,F|M
5,id,object,0.00,0.001,1000,1000,Sample: CU_1452838 | CU_4526886 | CU_4998329
6,online,bool,0.00,1.000,1000,1,False
7,profession,category,0.10,0.237,900,15,Analyst Programmer|Assistant Professor|Data Coordiator|Director of Sale...
8,start_date,datetime64[ns],0.00,0.007,1000,350,max=2018-12-30 00:00:00 | min=2018-01-01 00:00:00 | yr mean= 2018
9,surname,object,0.00,0.001,1000,1000,Sample: Gilday | Devargas | Oshima


---------
## Transitioning: Running the Contract Pipeline
Running the Contract Pipeline is very easy, because you have the Transitioning instance, you simply refresh your cannonical dataset<br>

How it works:
* Loads the new source dataset
* Loads the contracts parameterised intent (pipeline contract)
* Runs the intent against the dataset
* Returns the transitioned canonical

In [27]:
df = tr.refresh_clean_canonical()

tr.canonical_report(df, stylise=False)

,Attribute,dType,%_Null,%_Dom,Count,Unique,Observations
0,Unnamed:_0,int64,0.00,0.001,1000,1000,max=999 | min=0 | mean=499.5
1,age,float64,0.15,0.002,850,837,max=89.46799999999998 | min=20.053 | mean=47.4
2,balance,float64,0.00,0.002,1000,976,max=873.97 | min=29.54 | mean=182.72
3,forename,object,0.00,0.002,1000,997,Sample: Sahib | Vern | Riya
4,gender,category,0.00,0.626,1000,2,F|M
5,id,object,0.00,0.001,1000,1000,Sample: CU_7307739 | CU_5603121 | CU_3337438
6,online,bool,0.00,1.000,1000,1,False
7,profession,category,0.10,0.237,900,15,Analyst Programmer|Assistant Professor|Data Coordiator|Director of Sale...
8,start_date,datetime64[ns],0.00,0.007,1000,350,max=2018-12-30 00:00:00 | min=2018-01-01 00:00:00 | yr mean= 2018
9,surname,object,0.00,0.001,1000,1000,Sample: Viscia | Tubville | Fujikawa
